In [2]:
from expedia_scraper import parse
from datetime import datetime
from datetime import timedelta  
from random import randint
from time import sleep
from pandas import DataFrame
from pandas import date_range
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [6]:
def get_earliest_start_date(dow):    
    start_date = datetime.today()
    while start_date.weekday() != dow:
        start_date += timedelta(days= 1)
    return start_date
def create_flights(source,dest,start_date,los):
    flights = []    
    for date in dates:
        tup = (source, dest, format(date,'%d/%m/%Y'),format(date+ los,'%d/%m/%Y'))
        flights.append(tup)
    return flights

In [7]:
start_date = get_earliest_start_date(3) #Start on thursday
dates = date_range(start= start_date, periods = 4, freq = 'W', normalize = True,)
flights = create_flights('SIN','HKG',get_earliest_start_date(3),4)

In [8]:
results = []
for flight, (source, destination,dateout,datein) in enumerate(flights):        
    print("Doing {0} to {1} now...".format(dateout,datein))
    scrapes = parse(source, destination,dateout,datein)    
    for scrape in scrapes:
        scrape.update({'dateout': dateout,'datein':datein})
    results.extend(scrapes)
    #Random Sleep
    pause = randint(1,10)
    sleep(pause)    
    print('Sleeping for ' + str(pause) + ' seconds')    

Doing 04/08/2019 to 01/09/2019 now...
Sleeping for 9 seconds
Doing 11/08/2019 to 08/09/2019 now...
Sleeping for 8 seconds
Doing 18/08/2019 to 15/09/2019 now...
Sleeping for 6 seconds
Doing 25/08/2019 to 22/09/2019 now...
Sleeping for 6 seconds


In [9]:
#still needs work not sure why it requires 2 runs to work
df = DataFrame(results)
df['rownum'] = df['timings'].index.values.astype(int)
df['dep_time'] = None
df['arr_time'] = None

In [12]:
try:
    for i in df['rownum']:
        df['dep_time'].iloc[i] = df['timings'][i][0]['departure_time']
        df['arr_time'].iloc[i] = df['timings'][i][-1]['arrival_time'] 
except:
    try:
        for i in df['rownum']:
            df['dep_time'].iloc[i] = df['timings'][i][0]['departure_time']
            df['arr_time'].iloc[i] = df['timings'][i][-1]['arrival_time'] 
    except:
        pass

In [14]:
#Send to PG_SQL:
#Backup
df.to_csv("C:\\Users\\User\\Documents\\GitHub\\Expedia-Scraper\\{0}_SIN_to_HKG.csv".format(datetime.now().strftime("%Y_%m_%d")))